In [162]:
import pandas as pd
import re
import numpy
import pypika

In [16]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [68]:
def split_by(text, by = [","," & "," - ", "&amp;"]):
    text = [text]
    for char in by:
        text = flatten(map(lambda x: x.split(char), text))
    text = list(map(lambda x: x.strip(), text))
    text = list(filter(lambda x: x != "", text))
    return text

In [113]:
def extract_whd(text):
    if pd.isna(text):
        return text
    values = re.findall("\d+\.\d+", text)
    return list(map(float, values))

In [146]:
def extract_float(text):
    if pd.isna(text):
        return text
    value = re.findall("\d+[\.\d+]*", text)
    return float(value[0])

In [177]:
books = pd.read_csv("../../d/books.csv")

In [178]:
books['authors_arr'] = books['authors'].apply(lambda x: split_by(x))
authors = flatten(books['authors_arr'].values)
authors = list(map(lambda x: x.strip(), authors))
authors = list(set(authors))

In [179]:
books['authors_ids'] = books['authors_arr'].apply(lambda arr: [authors.index(author.strip()) for author in arr])

In [180]:
publishers = set(books['publisher'].values)
publishers = list(publishers)

books['publisher_id'] = books['publisher'].apply(lambda p: publishers.index(p))

In [181]:
titles = set(books['title'].values)
titles = list(titles)

books['book_id'] = books['title'].apply(lambda p: titles.index(p))

In [182]:
books['subjects_arr'] = books['subjects'].fillna("").apply(lambda x: split_by(x))

subjects = list(books['subjects_arr'].values)
subjects = flatten(subjects)
subjects = list(map(lambda x: x.strip(), subjects))
subjects = list(set(subjects))

In [183]:
books['subjects_ids'] = books['subjects_arr'].apply(lambda arr: [subjects.index(subject.strip()) for subject in arr])

In [184]:
whd = books.dimensions.apply(extract_whd)
books['width'] = whd.apply(lambda x: x[0] if type(x) == list else x)
books['height'] = whd.apply(lambda x: x[1] if type(x) == list else x)
books['depth'] = whd.apply(lambda x: x[2] if type(x) == list else x)

In [185]:
books['lexile'] = books['lexile'].apply(extract_float)
books['price'] = books['price'].apply(extract_float)

In [186]:
books = books[['id', 'book_id', 'isbn13', 'isbn10', 'price', 'pubyear', 'lexile', 'pages',
       'authors_ids', 'publisher_id', 'subjects_ids', 'width', 'height', 'depth']]

In [187]:
books.head()

,id,book_id,isbn13,isbn10,price,pubyear,lexile,pages,authors_ids,publisher_id,subjects_ids,width,height,depth
0,7,151,9781592211517,1592211518,23.95,2006,NaN,146.0,"[118, 577]",116,"[97, 269, 339, 443, 296, 82, 68, 30, 372, 204,...",8.30,5.30,0.8
1,474,439,9780945575931,945575939,1.99,1992,NaN,128.0,"[603, 745, 603]",121,"[290, 416, 59]",8.36,8.28,0.5
2,424,247,9781555838270,1555838278,1.99,2004,NaN,368.0,[541],79,"[317, 59, 474, 50, 311, 189]",5.40,8.50,0.8
3,236,243,9781555839970,1555839975,1.99,2007,NaN,288.0,"[538, 67]",79,"[59, 474, 50, 311]",6.00,9.00,0.7
4,229,370,9781555837471,1555837476,1.99,2004,NaN,264.0,"[5, 428]",79,"[279, 133, 430, 254, 50, 72, 50, 425, 327]",5.40,8.50,0.6


In [163]:
from pypika import Table, Query

In [211]:
author_q = Query.into(Table('author'))
for i, author in enumerate(authors):
    author_q = author_q.insert((i, author))

publisher_q = Query.into(Table('publisher'))
for i, publisher in enumerate(publishers):
    publisher_q = publisher_q.insert((i, publisher))

subject_q = Query.into(Table('subject'))
for i, subject in enumerate(subjects):
    subject_q = subject_q.insert((i, subject))

book_q = Query.into(Table('book'))
for i, book in enumerate(titles):
    book_q = book_q.insert((i, book))

parameter_q = Query.into(Table('edition_parameter')) \
    .insert((0, 'lexile')) \
    .insert((1, 'pages')) \
    .insert((2, 'width')) \
    .insert((3, 'height')) \
    .insert((4, 'depth'))

#str(author_q) + "\n\n" + str(publisher_q)

In [222]:
edition_q = Query.into(Table('edition'))
book_to_subject_q = Query.into(Table('book_to_subject')).columns(['book_id', 'subject_id'])
book_to_author_q = Query.into(Table('book_to_author')).columns(['book_id', 'author_id'])
parameter_value_q = Query.into(Table('edition_parameter_value')).columns(['parameter_id', 'edition_id', 'value'])

for index, row in books.iterrows():
    edition_q = edition_q.insert((
        index,
        row['isbn10'],
        row['isbn13'],
        row['book_id'],
        row['publisher_id'],
        row['pubyear'],
        row['price'] if not pd.isna(row['price']) else 0
    ))
    
    for s in row['subjects_ids']:
        book_to_subject_q = book_to_subject_q.insert((row['book_id'], s))

    for a in row['authors_ids']:
        book_to_author_q = book_to_author_q.insert((row['book_id'], a))

    for param_id, name in enumerate(['lexile', 'pages', 'width', 'height', 'depth']):
        if pd.isna(row[name]):
            continue
        parameter_value_q = parameter_value_q.insert((param_id, index, row[name]))

In [223]:
query = str(parameter_q) + ";\n\n" + str(author_q) + ";\n\n" + str(publisher_q) \
     + ";\n\n" + str(subject_q) + ";\n\n" + str(book_q) + ";\n\n" + str(edition_q) \
     + ";\n\n" + str(book_to_subject_q) + ";\n\n" + str(book_to_author_q) + ";\n\n" \
     + str(parameter_value_q) + ";"

text_file = open("books_data.sql", "w")
text_file.write(query)
text_file.close()